In [156]:
import pandas as pd
import re

In [157]:
df = pd.read_csv('datasets/amazon/amazon.csv')

In [158]:
df.rename(columns={'uniq_id': 'id', 'product_name': 'name', 'manufacturer': 'brand', 'number_available_in_stock': 'stock', 'average_review_rating': 'rating', 'amazon_category_and_sub_category': 'category', 'customer_reviews': 'reviews'}, inplace=True)
df.drop(['number_of_reviews', 'number_of_answered_questions', 'customers_who_bought_this_item_also_bought', 'product_information', 'product_description', 'items_customers_buy_after_viewing_this_item', 'customer_questions_and_answers', 'sellers'], inplace=True, axis=1)

In [159]:
# PRICE
df['price'] = df['price'].str.replace('£','') # remove £ - price
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df.round({"price": 2})

,id,name,brand,price,stock,rating,category,description,reviews
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,3.42,5 new,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,Product Description Hornby 2014 Catalogue Box ...,Worth Buying For The Pictures Alone (As Ever) ...
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,16.99,NaN,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,Size Name:Large FunkyBuys® Large Christmas Hol...,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...
2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,9.99,2 new,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,**Highly Recommended!** // 5.0 // 26 May 2015 ...
3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,39.99,NaN,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,I love it // 5.0 // 22 July 2013 // By\n \n...
4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,32.19,NaN,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,Product Description Hornby RailRoad 0-4-0 Gild...,Birthday present // 5.0 // 14 April 2014 // By...
...,...,...,...,...,...,...,...,...,...
9995,44d6967f083825a5de36ad4865a65bcd,Batman 1966 TV Series Action Figures - The Rid...,Mattel,22.95,5 new,5.0 out of 5 stars,Hobbies > Collectible Figures & Memorabilia > ...,DC 66 Batman Classic TV Series 6 Inch Riddler ...,Realistic // 5.0 // 31 Mar. 2014 // By\n \n...
9996,08f0747b6fc6687215ffb994c3a6fb32,"Star Wars Costume, Kids Stormtrooper Costume S...",Star Wars,39.99,NaN,4.0 out of 5 stars,Characters & Brands > Star Wars > Toys,NaN,... what I see my grandson us going to have fu...
9997,bf6cc073f8f24e6e338190fa16f6ee9d,Defiance Lawkeeper Metal Badge Prop Replica,Olde Scotland Yard Ltd.,43.99,3 new,5.0 out of 5 stars,Novelty & Special Use > Novelty > Accessories ...,*Includes 1x Badge with holder and chain *High...,Five Stars // 5.0 // 18 Dec. 2015 // By\n \...
9998,cd783d0b8b44e631b9788b203eaaefae,Justice League of America Series 3 Green Lante...,DC Comics,49.81,3 new,5.0 out of 5 stars,Hobbies > Collectible Figures & Memorabilia > ...,Designed by Ed BenesIt's here - the third seri...,The best sculpt in a while // 5.0 // 13 May 20...


In [160]:
# STOCK
df['stock'] = df['stock'].str.replace('new','') # remove 'new' - stock
df['stock'] = df['stock'].str.replace('used','') # remove 'used' - stock
df['stock'] = df['stock'].str.replace(u'\xa0', u'') #remove non-breaking space in Latin1 - stock
df['stock'] = pd.to_numeric(df['stock'], errors='coerce')

In [161]:
# RATING
df['rating'] = df['rating'].str.replace(' out of 5 stars','')
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

In [162]:
# CATEGORY
df['category'] = df['category'].str.split(' > ')

In [ ]:
# REVIEWS
df['reviews'] = df['reviews'].str.split('|')

In [ ]:
def parseDate(string):
    d = string.split()
    month = d[1]
    months = {'Jan.': 1, 'Feb.': 2, 'Mar.': 3, 'April': 4, 'May': 5, 'Jun.': 6, 'July': 7, 'Aug.': 8, 'Sept.': 9, 'Oct.': 10, 'Nov.': 11, 'Dec.': 12}
    month = months[month]
    date = d[2] + "-" + str(month) + "-" + d[0]
    return date

In [ ]:
def parseAuthor(string):
    try:
        s = re.findall(r'By (.+?) on', string)
        return s[0]
    except:
        return ""
    

In [ ]:
def splitreviews(row):
    reviews = row.reviews
    r = []
    try:
        for review in reviews:
            review = " ".join(review.split())
            rev = review.split(' // ')
            author = parseAuthor(rev[3])
            date = parseDate(rev[2])
            aux = {'author': author, 'rating': rev[1], 'date': date, 'title': rev[0], 'body': rev[4]}
            r.append(aux)
        return r
    except:
        return []

df['reviews'] = df.apply(splitreviews, axis=1)

In [ ]:
df['store'] = "amazon"

In [ ]:
df.to_csv('datasets/amazon/products_amazon.csv', encoding='utf-8', index=False)